In [ ]:
keras.__version__

In [ ]:
!pip install h5py==2.10.0
#!pip install tensorflow==1.14.0
!pip install keras==2.1.6


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [ ]:

#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
#tf.compat.v1.disable_eager_execution()
inception.graph = tf.get_default_graph()

225214464/225209952 [==============================] - 6s 0us/step


In [ ]:
# Get images


def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/ColabNotebooks/
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X


validation_data = []
for filename in os.listdir('Valid/'):
    validation_data.append(img_to_array(load_img('Valid/'+filename)))
validation_data = np.array(validation_data, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*validation_data))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*validation_data)[:,:,:,0]
Y_batch = rgb2lab(1.0/255*validation_data)[:,:,:,1:] / 128
color_me = color_me.reshape(color_me.shape+(1,))
V = ([color_me, color_me_embed], Y_batch)



In [ ]:
from keras import backend as K

def siren_in(x):
    x *= 1/30
    return tf.math.sin(x)  

In [ ]:
from keras.utils.generic_utils import get_custom_objects

get_custom_objects().update({'siren_in': Activation(siren_in)})

In [ ]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [ ]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation="siren_in", kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1), padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3),  activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3),  activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3),  activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3),activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [ ]:
embed_input = Input(shape=(1000,))
#activation="siren_in", kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1)
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation="siren_in", kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1), padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1),activation=tf.math.sin, kernel_initializer="he_uniform",  padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3),activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3),activation=tf.math.sin, kernel_initializer="he_uniform",padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3),activation=tf.math.sin, kernel_initializer="he_uniform", padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [ ]:
from keras.callbacks import History
from keras.callbacks import Callback


class SensitivitySpecificityCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 100 == 1:
            color_me = []
            for filename in os.listdir('Test/'):
                color_me.append(img_to_array(load_img('Test/'+filename )))
            color_me = np.array(color_me, dtype=float)
            gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
            color_me_embed = create_inception_embedding(gray_me)
            color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
            color_me = color_me.reshape(color_me.shape+(1,))


            # Test model
            output = self.model.predict([color_me, color_me_embed])
            output = output * 128

            # Output colorizations
            for i in range(len(output)):
                cur = np.zeros((256, 256, 3))
                cur[:,:,0] = color_me[i][:,:,0]
                cur[:,:,1:] = output[i]
                imsave("result/img_"+str(i)+ str(epoch) +".png", lab2rgb(cur))

In [ ]:
from keras.callbacks import History 

history = History()


# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 50

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)
        
model.compile(optimizer='Adam', loss="mse") #metrics=['accuracy']
with tf.device('/device:GPU:0'):
  model.fit_generator(image_a_b_gen(batch_size), epochs=1400, validation_data=V,
                    steps_per_epoch=3, callbacks=[SensitivitySpecificityCallback(), history])


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(50,1200),history.history["loss"][50:1200], "purple", label="loss")
plt.plot(np.arange(50,1200),history.history["val_loss"][50:1200], "brown", label="val_loss")
plt.title("Relu loss and validation loss over epochs")
plt.ylabel("MSE")
plt.xlabel("epoch (3 steps/epoch)")
plt.legend()

np.min(history.history["loss"])

In [ ]:
plt.plot(np.arange(50,150),history.history["loss"][50:150], "y")
plt.plot(np.arange(50,150),history.history["val_loss"][50:150], "pink")
plt.title("Siren loss and validation loss over epochs")
plt.ylabel("MSE")
plt.xlabel("epoch (2 steps/epoch)")

np.min(history.history["loss"])

In [ ]:
color_me = []
for filename in os.listdir('other/garde/'):
    color_me.append(img_to_array(load_img('other/garde/'+ filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result_z/ima_garde"+str(i)+".png", lab2rgb(cur))